# Load datasets

In [1]:
from datasets import load_dataset
import torch
ds = load_dataset("hungnm/vietnamese-medical-qa", split="train")

In [2]:
# print first 5 samples
for i in range(5):
    print(ds[i])

{'answer': 'Chào bạn,\nĐể trả lời câu hỏi trên, bác sĩ xin giải đáp như sau:\nRăng bạn hiện tại có mủ dưới lợi gây đau nhức nhiều. Bạn có thể đến phòng khám răng hàm mặt bệnh viện để được thăm khám, chụp phim và tư vấn cho bạn được chính xác\nTrân trọng!', 'question': 'Chào bác sĩ,\nRăng cháu hiện tại có mủ ở dưới lợi nhưng khi đau cháu sẽ không ngủ được (quá đau). Tuy nhiên chỉ vài ngày là hết mà thỉnh thoảng nó lại bị đau. Chị cháu bảo là trước chị cháu cũng bị như vậy chỉ là đau răng tuổi dậy thì thôi. Bác sĩ cho cháu hỏi đau răng kèm có mủ dưới lợi là bệnh gì? Cháu có cần đi chữa trị không? Cháu cảm ơn.'}
{'answer': 'Chào bạn,\nĐể trả lời câu hỏi trên, bác sĩ xin giải đáp như sau:\nTriệu chứng nốt mụn đỏ vùng dưới lưỡi, đau khi chạm vào gợi ý tình trạng mụn viêm vùng lưỡi, nếu nốt mụn không to thêm và tự hết trong 7-10 ngày thì bạn không cần quá lo lắng.\nTrong trường hợp nốt mụn to dần hoặc nốt mụn tồn tại trên 02 tuần không hết thì bạn cần đến bác sĩ để khám kiểm tra. Đối với các

In [3]:
import re

def clean_text(text):
    # Loại bỏ các cụm từ dư thừa
    patterns = [
        r"(?i)\b(chào|xin chào|kính chào)\s*(bác sĩ|bạn|em|anh|chị)?[:,!.]?",
        r"(?i)\b(để trả lời câu hỏi trên, bác sĩ xin giải đáp như sau)[:,]?",
        r"(?i)\b(để trả lời câu hỏi trên, bác sĩ giải đáp như sau)[:,]?",
        r"(?i)\b(bác sĩ cho (cháu|em|tôi) hỏi)[:,]?",
        r"(?i)\b(em muốn hỏi)[:,]?",
        r"(?i)\b(xin hỏi)[:,]?",
        r"(?i)\b(em cảm ơn|cháu cảm ơn|cháu xin cảm ơn bs)[:,.!]?",
        r"(?i)\b(trân trọng|thân ái|chúc bạn sức khỏe|thân mến)[:,.!]?"
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [4]:
def preprocess(example):
    input = clean_text(example["question"])
    response = clean_text(example["answer"])
    return {
        "instruction": "",
        "input": input,
        "output": response
    }

processed_dataset = ds.map(
    preprocess,
    remove_columns=["question", "answer"],
)

In [5]:
for i in range(5):
    print(processed_dataset[i])

{'instruction': '', 'input': 'Răng cháu hiện tại có mủ ở dưới lợi nhưng khi đau cháu sẽ không ngủ được (quá đau). Tuy nhiên chỉ vài ngày là hết mà thỉnh thoảng nó lại bị đau. Chị cháu bảo là trước chị cháu cũng bị như vậy chỉ là đau răng tuổi dậy thì thôi. đau răng kèm có mủ dưới lợi là bệnh gì? Cháu có cần đi chữa trị không?', 'output': 'Răng bạn hiện tại có mủ dưới lợi gây đau nhức nhiều. Bạn có thể đến phòng khám răng hàm mặt bệnh viện để được thăm khám, chụp phim và tư vấn cho bạn được chính xác'}
{'instruction': '', 'input': 'Em thấy mặt dưới, phía cuống lưỡi của mình có 2 nốt mụn nhỏ, đỏ xung quanh, ở giữa có nhân trắng, đau nhẹ khi dùng đầu lưỡi chạm vào. Đồng thời, phía cuống lưỡi mặt trên cũng có các nốt lớn nổi lên, không gây đau. cuống lưỡi nổi mụn nhỏ là dấu hiệu bệnh gì? Có phải em đang mắc bệnh gì không hay chỉ bị nhiệt miệng bình thường? bác sĩ.', 'output': 'Triệu chứng nốt mụn đỏ vùng dưới lưỡi, đau khi chạm vào gợi ý tình trạng mụn viêm vùng lưỡi, nếu nốt mụn không to 

In [6]:
full_dataset = processed_dataset.train_test_split(test_size=0.05, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']
 
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 8868
})
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 467
})


# Load model

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training

model_name = "binhphap5/gpt2-vietnamese-medium-instruct-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:0",
    torch_dtype = torch.bfloat16,
    # quantization_config = bnb_config,
)

model.enable_input_require_grads() 

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# tokenizer.pad_token = tokenizer.eos_token
# model = prepare_model_for_kbit_training(model)

In [8]:
# Total parameters and trainable parameters.
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50258, bias=False)
)
354,824,192 total parameters.
354,824,192 

# Build LoRA config

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    fan_in_fan_out = True,
    inference_mode = False
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() 

trainable params: 4,325,376 || all params: 359,149,568 || trainable%: 1.2043


In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50258, 1024)
        (wpe): Embedding(1024, 1024)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=3072, nx=1024)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
    

# Preprocessing

In [11]:
def preprocess_function(example):
    """
    Formatting function with clear delimiters and handling of empty inputs.
    """
    # instruction = example['instruction'].strip()
    # Handle empty or None input gracefully
    input_text = example["input"].strip() if example["input"] else ""
    output = example["output"].strip()

    # Format with clear separators
    # if input_text:
    text = f"### Instruction:\nHãy trả lời câu hỏi với mô tả sau.\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    # else:
    #     text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}\n\n"
    return text

In [12]:
# data collator for causal LM
from trl import DataCollatorForCompletionOnlyLM

response_template = "### Response:\n"
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=response_template,
)

# Training

In [13]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    # Paths & Datasets
    output_dir="gpt2-vietnamese-medium-instruct-medical-qa",    
    logging_dir="logs",                  
    
    # Truncation 
    max_length=tokenizer.model_max_length,
    
    per_device_train_batch_size=4,       
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,

    # Optimization & LR Scheduling
    learning_rate=5e-5,
    weight_decay=0.03,
    num_train_epochs=10,
    lr_scheduler_type="cosine",

    # Evaluation / Checkpoint
    eval_strategy="epoch",              
    save_strategy="epoch",              
    logging_strategy="epoch",           
    save_total_limit=1,

    # Best‑model selection
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    optim="paged_adamw_32bit",
    gradient_checkpointing=True
)

In [14]:
trainer = SFTTrainer(
    model=model,                         
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    args=sft_config,                     
    processing_class=tokenizer,
    formatting_func=preprocess_function,
    data_collator=data_collator,
)

Applying formatting function to train dataset:   0%|          | 0/8868 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/8868 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8868 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8868 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1184 > 1024). Running this sequence through the model will result in indexing errors


...

Converting eval dataset to ChatML:   0%|          | 0/467 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/467 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/467 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/467 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# # get dataloader in trainer
# data_loader = trainer.get_train_dataloader()
# for batch in data_loader:
#     # print an example
#     print(batch["input_ids"][0].shape)
    

In [16]:
history = trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.015600,2.881226
2,2.863600,2.821907
3,2.803800,2.790011
4,2.759100,2.768887
5,2.728000,2.756187
6,2.703000,2.749101
7,2.686000,2.743526
8,2.674900,2.741616
9,2.665600,2.740715


c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\trl\trainer\utils.py:145: UserWarning: Could not find response key `### Response:
` in the following instance: ### Instruction:
Hãy trả lời câu hỏi với mô tả sau.

### Input:
Cho em hỏi sau khi quan hệ không bảo vệ em đã dùng thuốc tránh thai, nhưng thời gian cũng không lâu trong ngày hôm đó, em lại quan hệ không bảo vệ lần nữa, vậy thuốc tránh thai ban đầu em dùng có tác dụng với lần sau không ạ? Thanh Thúy (1996) tôi muốn hỏi sau 10 ngày quan hệ còn có thể uống thuốc tránh thai được không ạ? Mong bác sĩ tư vấn giúp tôi, cảm ơn bác sĩ. Cho em hỏi là em quan hệ tới giờ đã 5 ngày. Có loại thuốc tránh thai nào dùng được cho trường hợp của em nữa không ạ, em chưa muốn có con giờ. Cảm ơn bác sĩ Em có uống thuốc tránh thai 72h vào ngày 4/3 nhưng đến ngày 5/3 em quan hệ mà không dùng biện pháp tránh thai thì khả năng có thai cao không ạ. Cảm ơn bác sĩ! Em uống thuốc tránh thai khẩn cấp loại 72h và quan hệ không an toàn sau đó 1 ngày thì

In [18]:
torch.cuda.empty_cache()

In [ ]:
# push to hub
trainer.push_to_hub("binhphap5/gpt2-vietnamese-medium-instruct-medical-qa")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/gpt2-vietnamese-medium-instruct-medical-qa/commit/54dcbfd826e405538f0a263c22b702cbe5c06b5d', commit_message='binhphap5/gpt2-vietnamese-medium-instruct-medical-qa', commit_description='', oid='54dcbfd826e405538f0a263c22b702cbe5c06b5d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/gpt2-vietnamese-medium-instruct-medical-qa', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/gpt2-vietnamese-medium-instruct-medical-qa'), pr_revision=None, pr_num=None)

In [37]:
template = """### Instruction:
Hãy trả lời câu hỏi với mô tả sau.

### Input:
{}

### Response:
{}"""

input = 'Cơ thể của tôi cảm thấy nhức khi ngủ dậy vào buổi sáng, đây có thể là triệu chứng gì ?'
response = ''
prompt = template.format(input, response)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

### Instruction:
Hãy trả lời câu hỏi với mô tả sau.

### Input:
Cơ thể của tôi cảm thấy nhức khi ngủ dậy vào buổi sáng, đây có thể là triệu chứng gì?

### Response:
Với biểu hiện trên bạn nên khám bác sĩ chuyên khoa về thần kinh để xác định chính xác nguyên nhân và điều trị phù hợp!<|endoftext|>


In [30]:
metrics = trainer.evaluate()

In [31]:
metrics

{'eval_loss': 2.740715265274048,
 'eval_runtime': 4.6827,
 'eval_samples_per_second': 99.729,
 'eval_steps_per_second': 24.986}

In [32]:
torch.cuda.empty_cache()

# Calculate perplexity

In [33]:
import math
perplexity = math.exp(metrics["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 15.50


In [ ]:
# # compute rouge score
# from datasets import load_metric
# rouge = load_metric("rouge")
# predictions = [tokenizer.decode(pred, skip_special_tokens=True) for pred in outputs]
# rouge.add_batch(predictions=predictions, references=[response] * len(predictions))
# rouge_scores = rouge.compute()